In [1]:
from optuna.samplers import TPESampler
import optuna
from sklearn.model_selection import  StratifiedKFold,cross_val_score
from sklearn.metrics import roc_auc_score,roc_curve
def ks(estimator,X, y):
    y_predicted = estimator.predict_proba(X)[:,1]
    #label = y_true.get_label()
    fpr,tpr,thres = roc_curve(y,y_predicted,pos_label=1)
    return abs(fpr - tpr).max()

In [3]:
import lightgbm as lgb

In [ ]:
lgb.cv()

In [ ]:
lgb.LGBMClassifier()

In [ ]:
#这里使用一种以最大化测试集上KS值为目标的调参方式，这样可以兼顾模型的区分度和鲁棒性。
sampler = TPESampler(seed=10) 
def objective(trial):
    lgbm = lgb.cv(
    verbosity=-1,
    boosting_type= 'gbdt',
    lambda_l2=trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    learning_rate=trial.suggest_loguniform('learning_rate', 1e-8, 0.1),
    n_estimators=trial.suggest_int('n_estimators', 400, 800),
    feature_fraction=trial.suggest_uniform('feature_fraction', 0.4, 1.0),
    bagging_fraction= trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
    bagging_freq=trial.suggest_int('bagging_freq', 1, 7),
    min_child_samples=trial.suggest_int('min_child_samples', 5, 100),
    max_depth = trial.suggest_int('max_depth', 3, 8))   
    gbm = lgbm.fit(df_train_all_sel1,df_train_y)    
    return ks(gbm,df_test_all_sel1,df_test_y)

study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)
params = study.best_params

In [ ]:
np.save('params_lgb.npy', params)

In [ ]:
params_lgb = np.load('/home/aistudio/work/任务6/params_lgb.npy', allow_pickle=True).item()

In [ ]:
nfold = 5
gc.collect()
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2022)

In [ ]:
train_pre = np.zeros(train.shape[0])
test_pre = np.zeros((test.shape[0], nfold))

i = 1
for train_index, valid_index in skf.split(train, y_train, ):
    print('='*50)
    print("\nfold {}".format(i))
    train_df = lgb.Dataset(train.iloc[train_index], y_train.iloc[train_index],)
    val_df = lgb.Dataset(train.iloc[valid_index], y_train.iloc[valid_index],)

    clf = lgb.train(params_lgb, train_df, num_boost_round=20,
    valid_sets = [val_df], verbose_eval=5, early_stopping_rounds=10)

    train_pre[valid_index] = clf.predict(train.iloc[valid_index], num_iteration=clf.best_iteration)

    test_pre[:, i-1] = clf.predict(
        test, num_iteration=clf.best_iteration
    )
    i += 1

print("\n train dataset AUC:{:<0.2f}".format(roc_auc_score(y_train, train_pre)))